## Задание 1. (10 баллов)

Дообучите языковую модель на датасете инструкций, используя LoRA. Проверьте, что дообученная модель отличается от изначальной - сгенерируйте продолжения для одних и тех же промптов и сравните результаты.

Вы можете взять за основу код семинара PEFT, изменив датасет цитат на датасет инструкций (можно просто скопировать из семинара про General_instruct_fine-tuning).
Можно использовать alpaca_dataset, датасет Dolly 2 или переведенный датасет (или все вместе).
Важно использовать модель с большим количеством параметров (относительно семинара по General instruct fine-tuning).
Размер модели должен быть как минимум 3 млрд параметров.  
**Нужно использовать модель, которую мы не разбирали на семинаре (OPT-2.7b, OPT-6.7b). Найдите новую модель на huggingface hub.**



In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.gitи
!pip install peft
!pip install zstandard jsonlines
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 67.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to bu

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch
from peft import PeftModel, PeftConfig, LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True # уменьшила с 8 до 4, иначе не хватало памяти
    )

Я выбрала модель EleutherAI/gpt-j-6B (https://huggingface.co/EleutherAI/gpt-j-6b)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "lightblue/suzume-llama-3-8B-multilingual",
    quantization_config=quantization_config,
    cache_dir='./models'
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("lightblue/suzume-llama-3-8B-multilingual")

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
for param in model.parameters():
  param.requires_grad = False
  if param.ndim == 1:
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
for name, module in model.named_modules():
    print(name)


model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.self_attn.rotary_emb
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.rotary_emb
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.

In [ ]:
config = LoraConfig(
    r=32, # внутренняя размерность адаптера, основной параметр
    target_modules=["q_proj", "k_proj", "v_proj", 'o_proj'], # к каким слоям добавлять адаптеры (подробнее выше)
    lora_alpha=32,

    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 27262976 || all params: 4567863296 || trainable%: 0.5968430803057028


In [ ]:
import transformers
from datasets import load_dataset

dataset = load_dataset("IlyaGusev/ru_turbo_alpaca", split='train[:40%]')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for IlyaGusev/ru_turbo_alpaca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/IlyaGusev/ru_turbo_alpaca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/29822 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'alternative_output', 'label', 'all_labels', 'agreement', 'overlap'],
    num_rows: 11929
})

In [ ]:
dataset[:5]['instruction']

['Опишите, как сделать горшок из глины.',
 'Опиши процесс изготовления торта. Укажи ингредиенты, необходимые для этого.',
 'Напиши аналог данной пословицы на современный лад: "Не говори гоп, пока не перепрыгнешь".',
 'Представьте, что вы занимаетесь дизайном главной страницы для нового сайта. Опишите, что вы хотели бы видеть на этой странице.',
 'Напиши рассказ о своем детстве и самом смешном инциденте, который произошел с тобой.']

In [ ]:
dataset[:5]['output']

['Для изготовления горшка из глины сначала необходимо взять глину и размешать ее водой до состояния пластилина. Затем нужно сформировать глиняный комок нужной формы и тщательно откачать влагу из глины, чтобы она стала твердой. Горшок нужно обжечь в духовке при высокой температуре, чтобы он стал прочным и непроницаемым.',
 'Для изготовления торта необходимы следующие ингредиенты: мука, сахар, яйца, молоко, сливочное масло, разрыхлитель, ванилин. Сначала нужно смешать муку с сахаром и добавить яйца, молоко, разрыхлитель и ваниль. Затем тесто надо хорошо перемешать и разделить на несколько частей. Каждую часть нужно запекать в духовке до готовности. После этого торт надо собрать, сложив слои пирога вместе с начинкой и украшениями.',
 '"Не показывай свои способности, пока не будет нужно."',
 'Я бы хотел, чтобы главная страница была простой, но информативной. На ней должны быть ясно выделены основные разделы и функции сайта, а также действительно привлекательный визуальный дизайн. Я бы испо

In [ ]:
dataset = dataset.map(lambda sample: {"text": sample['instruction'] + ' ' + sample['output']})

Map:   0%|          | 0/11929 [00:00<?, ? examples/s]

In [ ]:
# выдается ошибка, так как я перезапускала гугл коллаб несколько раз и раннее удаляла "label"
dataset = dataset.remove_columns("label")

ValueError: Column name ['label'] not in the dataset. Current columns in the dataset: ['instruction', 'input', 'output', 'alternative_output', 'all_labels', 'agreement', 'overlap', 'text']

In [ ]:
def tokenize_function(samples):
    return tokenizer(samples['text'], padding=True, truncation=True, max_length=512)

In [ ]:
dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/11929 [00:00<?, ? examples/s]

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=400,
        learning_rate=1e-3,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False, )
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
model.config.use_cache = False

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
1,1.837000
2,1.834300
3,1.544700
4,1.561000
5,1.674000
6,1.775900
7,2.164700
8,1.644400
9,1.301900
10,2.379000


TrainOutput(global_step=400, training_loss=1.4754320047795773, metrics={'train_runtime': 3731.2788, 'train_samples_per_second': 0.429, 'train_steps_per_second': 0.107, 'total_flos': 3.0298737002840064e+16, 'train_loss': 1.4754320047795773, 'epoch': 0.134126917595775})

In [ ]:
model.save_pretrained('suzume_lora')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

In [ ]:
peft_model_id = "suzume_lora"

quantization_config = BitsAndBytesConfig(
        load_in_4bit=True
    )
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="lightblue/suzume-llama-3-8B-multilingual",
                                             return_dict=True,
                                             quantization_config=quantization_config,
                                             device_map='auto'
                                            )
tokenizer = AutoTokenizer.from_pretrained("lightblue/suzume-llama-3-8B-multilingual")

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### **Генерация ответов**

In [ ]:
def generate(text, tokenizer, model):
    batch = tokenizer(text, return_tensors='pt').to('cuda')
    output_tokens = model.generate(**batch, max_new_tokens=300, temperature=0.1, do_sample=True, no_repeat_ngram_size=3)

    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

In [ ]:
print(generate("С чем едят блины?",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


С чем едят блины? Блины могут быть приготовлены из различных ингредиентов, в зависимости от того, какие продукты вы предпочитаете. Вот несколько вариантов:

1. **Овсяные блинчики**: Для этого смешайте овсяные хлопья с молоком, яйцами, солью и сахаром. Добавьте немного муки, чтобы блины не были слишком жидкими. Обжаривайте их на сковороде, как обычные блины.

2. **Банановые блины**: Замочите бананы в молоке, добавьте яйца, сахар, соль и немного мука. Обратите внимание, что банановые блинцы могут получиться более жидкими, чем традиционные.

3. **Корзинные бляшики**: Для этих бляшек смешайте муку с яйцами и молоком. Дополнительные ингридиенты, такие как сахар, ванильный сахар, можно добавить в тесто. Обычно, корзинным бляшим не добавляют сахара, так как они сладкие сами по себе.

4. **Сырные блины (поффеты)**: Для сырных блинов смешайте творог с яйцом, добавив немного муку. Объедините смесь с молочным соусом или сметаной, чтобы получились мягкие и


In [ ]:
print(generate("Расскажи о векторном поле",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Расскажи о векторном поле, если оно существует, и о его свойствах.](/math/vectors/vector_field.html)

---

Векторное поле — это математическое понятие, которое описывает способность векторного поля изменять векторы в каждой точке пространства. Векторное пространство — это множество векторов, а векторное множественное — это способность множества векторных пространств изменять их векторные операции. Важно понимать, что векторная операция — это операция, которая преобразует вектор в вектор, но не обязательно сохраняет его длину и направление.

Векторные поля используются в различных областях математики и физики, включая дифференциальную геометрию, топологию, анализ и физику. Они позволяют описывать и аналлизировать различные физические и геометричные явления, такие как потоки жидкости, электрические поля, гравитационные поля и многие другие.

Вот несколько ключевых свойств векторной поля:

1. **Векторная функция**: Векторная поля часто ассоциируются с векторными функциями, которые в каждо

In [ ]:
print(generate("Придумай название для книги ужасов",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Придумай название для книги ужасов, основываясь на описании: "В небольшом городке, где каждый знает друг друга, начинается серия странных и таинственных событий. Люди начинают исчезать, а те, кто остается, сталкиваются с невероятными и пугающими явлениями. В центре всех этих событиий оказывается молодая женщина, которая пытается разгадать тайну и спасти город от неизвестной угрозы."](/books/strange-occurrences)  
[Присоединяйтесь к нашему сообществу](/community)  
Новости и обзоры: [Последние новости](/news)  
Обратите внимание, что все материалы на сайте предоставлены в качестве обзора и анализа. Ваши комментарии и отзывы по-прежнему приветствуются.](


### **PeftModel**

In [ ]:
model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
print(generate("С чем едят блины?",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


С чем едят блины? Блины могут быть приготовлены с различными ингредиентами, в зависимости от предпочтений. Некоторые популярные ингрединты для блинов включают молоко, яйца, муку, масло, сахар, ванильный сахар, соль и перец. Также могут быть добавлены различные фрукты, ягоды, орехи, шоколад и другие ингредиенты. Блину можно подавать с различным соусом, такими как сметана, фруктовый соус или шоколадки. Также можно подавить блину с различной заправкой, такой как сыворотка, йогурт или сметаной. В зависимности от того, какие ингридиенты вы используете, блина может быть более сладкой или соленой. Также может быть добавлено различные специи, такие как имбирь, гвоздика или кориандр, чтобы добавить аромат. В целом, блины могут подаваться с различного рода и вкусом, в зависимости от того какие индредиенты вы выберете. Они могут быть очень вкусными и наслаждаться их вкусом. Также они могут быть полезными для здоровья, так как они содержат много витаминов и минералов. Блинчики могут быть готовы в 

In [ ]:
print(generate("Расскажи о векторном поле",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Расскажи о векторном поле. Векторное поле - это математическое пространство, в котором элементы - это векторы, а операции - это сложение и умножение векторов. Векторов можно перемещать, вращать и складывать, а также умножать на числа. В векторное пространство можно работать с операциями, которые не имеют смысла в обычном пространстве, например, умножить вектор на число и получить вектор. Вектора можно также умноьнять и делить, но только если они не равны нулю. В пространстве можно работать и с операцией умножения вектора на вектор, которая не является коммутативной, то есть не сохраняет порядок элементов. В некоторых случаях векторные поля могут быть нелинейными, тоесть операции не сохраняют линейность. В таких случаями векторная операция может быть не дистрибутивной, что означает, что порядок операций может изменить результат. В общем случае векторный поле может быть очень сложным и нелинеарным, и требует специальных методов для работы с ним. Важно понимать, что вектор может быть пред

In [ ]:
print(generate("Придумай название для книги ужасов",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Придумай название для книги ужасов. "Тень в комнате". "Охота на ведьму". "Затеревшаяся душа". "Сердце дракона". "Тайный сад". "Ведьма из леса". и т.д. "Огонь в душе". "Невестка из будущего". "Кровь и камень". "Мертвые города". "Чумный город". "Похищение". "Дом на краю света". "Бесконечная ночь". "Где-то в глубине". "Рыцарь тьмы". "Исповедь драконя". "Последний приют". "Старая ведьма". "Шептун". "Лесные драконы". "Убийца". "Проклятие". и другие. Эти названия могут быть использованы для создания атмосферы и настроения в книге. Они могут вызвать интерес у читателя и заставить его задуматься о теме книги. Названия должны быть уникальными и запоминающимися, чтобы читатель мог легко вспомнить их при чтении книги. Они также должны соответствовать содержанию книги и не быть слишком очевидными или предсказуемыми. Наконец, они должны быть интересными для читателя, чтобы он мог наслаждаться чтением книги


PeftModel справилась лучше: её ответы более конкретные. Она хорошо справилась с "придумыванием". Но в целом обе генерируют ответы неплохо.